The following predicts the probability of crime occurance on:
1. days in the week
2. hours in the day 
3. Month in the year 
To do this the Random forest model was adopted to categorize and predict the probability of cirme occuring on different days, hours and month.   

In [2]:
#import the necessary libraries 
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np

In [3]:
#import the dataset 
crime_data = pd.read_csv('https://raw.githubusercontent.com/Pelumioluwa/MBAN6110Class/main/Data/Toronto%20Police%20Shooting%20and%20Firearm%20Discharge.csv')
crime_data

,X,Y,OBJECTID,EVENT_UNIQUE_ID,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DOW,OCC_DOY,OCC_DAY,...,OCC_TIME_RANGE,DIVISION,DEATH,INJURIES,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-79.234955,43.781528,1,GO-2004397105,2004/05/15 04:00:00+00,2004,May,Saturday,136,15,...,Night,D41,0,0,142,Woburn North,137,Woburn (137),-79.234955,43.781528
1,-79.618218,43.733547,2,GO-2004600109,2004/07/26 04:00:00+00,2004,July,Monday,208,26,...,Evening,D23,0,0,1,West Humber-Clairville,1,West Humber-Clairville (1),-79.618218,43.733547
2,-79.518759,43.769157,3,GO-2004311851,2004/05/14 04:00:00+00,2004,May,Friday,135,14,...,Afternoon,D31,1,1,24,Black Creek,24,Black Creek (24),-79.518759,43.769157
3,-79.378870,43.660665,4,GO-2004736004,2004/12/19 05:00:00+00,2004,December,Sunday,354,19,...,Night,D51,1,0,168,Downtown Yonge East,75,Church-Yonge Corridor (75),-79.378870,43.660665
4,-79.212435,43.812075,5,GO-2004303455,2004/04/12 04:00:00+00,2004,April,Monday,103,12,...,Evening,D42,0,0,146,Malvern East,132,Malvern (132),-79.212435,43.812075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,-79.362941,43.771764,5703,GO-20221038418,2022/06/02 04:00:00+00,2022,June,Thursday,153,2,...,Night,D33,0,1,53,Henry Farm,53,Henry Farm (53),-79.362941,43.771764
5703,-79.402702,43.643920,5704,GO-20221362949,2022/07/17 04:00:00+00,2022,July,Sunday,198,17,...,Night,D14,1,1,164,Wellington Place,77,Waterfront Communities-The Island (77),-79.402702,43.643920
5704,-79.196217,43.786621,5705,GO-2022336423,2022/02/21 05:00:00+00,2022,February,Monday,52,21,...,Night,D43,0,0,135,Morningside,135,Morningside (135),-79.196217,43.786621
5705,-79.443993,43.658083,5706,GO-20221765442,2022/09/11 04:00:00+00,2022,September,Sunday,254,11,...,Night,D11,0,2,83,Dufferin Grove,83,Dufferin Grove (83),-79.443993,43.658083


First step is to predict the probability for crime occurance on the days in the week 
1. Convert day of the week into label 
2. Convert the categorical variable into dummy data and one-encode it 
3. Scale the numerical variables (day of the year,day of the month, hour of the day)
4. Make date time the index



In [4]:
#Separate the different numerical features, categorical features, label and target  
categorical_features = ['OCC_MONTH','OCC_TIME_RANGE']
numerical_features = ['OCC_DOY','OCC_DAY','OCC_HOUR']
target = ['OCC_DOW']
label = ['OCC_DOW']


In [5]:
#define label encoder and convert the days of week into labels
label_encoder_trans = LabelEncoder()
crime_data['OCC_DOW'] = label_encoder_trans.fit_transform(crime_data['OCC_DOW'])


In [6]:
crime_data[target]


,OCC_DOW
0,2
1,1
2,0
3,3
4,1
...,...
5702,4
5703,3
5704,1
5705,3


In [7]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data[target],
                                                    test_size = 0.3, random_state=1234)

In [8]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])

In [9]:
#build the preprocessor 
preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),
                                               ('numerical',numerical_transformer,numerical_features) ])

In [10]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [11]:
#fit the model
random_forest.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OCC_MONTH',
                                                   'OCC_TIME_RANGE']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['OCC_DOY', 'OCC_DAY',
                                                   'OCC_HOUR'])])),
                ('regressor', RandomForestClassifier())])

In [12]:
# Predict the probabilities of crime occurrence for each day of the week in the test set
day_probabilities = random_forest.predict_proba(X_test)

In [13]:
# Display the probabilities for each day of the week in the test set
day_names = label_encoder_trans.inverse_transform(random_forest.classes_)
probabilities_df = pd.DataFrame(day_probabilities, columns=day_names)
print(probabilities_df)

        Friday    Monday  Saturday    Sunday  Thursday   Tuesday  Wednesday
0     0.380000  0.030000  0.110000  0.110000  0.050000  0.220000   0.100000
1     0.110000  0.000000  0.040000  0.290000  0.000000  0.550000   0.010000
2     0.010000  0.000000  0.186667  0.523333  0.020000  0.153333   0.106667
3     0.460000  0.131667  0.060000  0.130000  0.068333  0.003333   0.146667
4     0.047619  0.371667  0.033333  0.260000  0.085714  0.110000   0.091667
...        ...       ...       ...       ...       ...       ...        ...
1708  0.046667  0.010000  0.263333  0.291333  0.010000  0.378667   0.000000
1709  0.060000  0.490000  0.030000  0.030000  0.230000  0.010000   0.150000
1710  0.457333  0.000000  0.040000  0.462667  0.000000  0.040000   0.000000
1711  0.146000  0.049083  0.050000  0.064917  0.440000  0.000000   0.250000
1712  0.098605  0.043767  0.000000  0.140000  0.000000  0.717628   0.000000

[1713 rows x 7 columns]


In [14]:
#import evaluation metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score


In [15]:
predicted_classes = day_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to day names
predicted_day_names = label_encoder_trans.inverse_transform(predicted_classes)

In [16]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print("Accuracy:", accuracy)

Accuracy: 0.20198482194979567


In [17]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[44 31 36 38 41 23 16]
 [28 48 29 44 34 26 31]
 [39 32 74 76 35 26 24]
 [46 35 43 64 42 36 20]
 [30 34 37 32 38 23 16]
 [31 29 36 27 30 47 26]
 [27 32 39 39 23 25 31]]


In [18]:
# Calculate classification report (includes precision, recall, and F1-score for each class)
classification_rep = classification_report(y_test, predicted_classes, target_names=label_encoder_trans.classes_)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

      Friday       0.18      0.19      0.19       229
      Monday       0.20      0.20      0.20       240
    Saturday       0.25      0.24      0.25       306
      Sunday       0.20      0.22      0.21       286
    Thursday       0.16      0.18      0.17       210
     Tuesday       0.23      0.21      0.22       226
   Wednesday       0.19      0.14      0.16       216

    accuracy                           0.20      1713
   macro avg       0.20      0.20      0.20      1713
weighted avg       0.20      0.20      0.20      1713



In [19]:
# Calculate AUC-ROC (requires one-hot encoded y_test and day_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
auc_roc = roc_auc_score(y_test_one_hot, day_probabilities, multi_class='ovr')
print("AUC-ROC:", auc_roc)

AUC-ROC: 0.5608749413639351


In [20]:
#get the day with the high mean of probability 
probabilities_df.describe(include='all')

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
count,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000
mean,0.144451,0.140670,0.166568,0.177424,0.131570,0.121587,0.117730
std,0.186148,0.193353,0.210420,0.213027,0.183056,0.181127,0.169317
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.010000,0.010000,0.020000,0.020000,0.010000,0.005833,0.010000
50%,0.062000,0.050000,0.075000,0.083833,0.050000,0.039556,0.040000
75%,0.210000,0.190000,0.232167,0.260000,0.170000,0.153333,0.150000
max,0.910000,0.970000,0.930000,0.970667,0.910000,0.973333,0.877262


In [21]:
#get the day with the highest sum of probability 
day_sum_prob = round(probabilities_df.sum(),2)
print(day_sum_prob)
df_day_sum_prob = pd.DataFrame(day_sum_prob)
df_day_sum_prob.reset_index()
#renaming the column probabilities 
df_day_sum_prob = df_day_sum_prob.rename(columns={0: 'Cum_prob'})

Friday       247.44
Monday       240.97
Saturday     285.33
Sunday       303.93
Thursday     225.38
Tuesday      208.28
Wednesday    201.67
dtype: float64


In [22]:
df_day_sum_prob

,Cum_prob
Friday,247.44
Monday,240.97
Saturday,285.33
Sunday,303.93
Thursday,225.38
Tuesday,208.28
Wednesday,201.67


From the above, we see the day of the week with the highest mean probability is Sunday (0.176272) and Saturday (0.166053). Therefore, the weekends are the days in the week the NCOP should be on stand for as they have the highest probability of crime occurance 

Next we predict the probability for the month 

In [23]:
#label the month 
crime_data['OCC_MONTH'] = label_encoder_trans.fit_transform(crime_data['OCC_MONTH'])
target = crime_data['OCC_MONTH']

In [24]:
#Separate the different numerical,categorica features, target  
categorical_features = ['OCC_DOW','OCC_TIME_RANGE']
numerical_features = ['OCC_DOY','OCC_DAY','OCC_HOUR']
label = ['OCC_MONTH']
target = ['OCC_MONTH']

In [25]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data[target],
                                                    test_size = 0.3, random_state=1234)

In [26]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])

In [27]:
#building the preprocessor
preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),
                                               ('numerical',numerical_transformer,numerical_features) ])

In [28]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [29]:
#fit the model
random_forest.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OCC_DOW',
                                                   'OCC_TIME_RANGE']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['OCC_DOY', 'OCC_DAY',
                                                   'OCC_HOUR'])])),
                ('regressor', RandomForestClassifier())])

In [30]:
# Predict the probabilities of crime occurrence for each month in the test set
month_probabilities = random_forest.predict_proba(X_test)

In [31]:
# Display the probabilities for each month in the test set
month_names = label_encoder_trans.inverse_transform(random_forest.classes_)
month_probabilities_df = pd.DataFrame(month_probabilities, columns=month_names)
print(month_probabilities_df)

      April  August  December  February  January  July  June  March   May  \
0      0.12    0.02      0.00      0.22     0.07  0.00  0.05   0.51  0.01   
1      0.00    0.93      0.05      0.00     0.01  0.00  0.01   0.00  0.00   
2      0.00    0.01      0.37      0.00     0.00  0.01  0.03   0.00  0.00   
3      0.21    0.06      0.00      0.03     0.12  0.00  0.15   0.00  0.41   
4      0.01    0.12      0.00      0.00     0.00  0.01  0.02   0.00  0.00   
...     ...     ...       ...       ...      ...   ...   ...    ...   ...   
1708   0.00    0.00      0.03      0.00     0.00  0.03  0.00   0.00  0.00   
1709   0.07    0.03      0.00      0.00     0.13  0.08  0.07   0.10  0.42   
1710   0.01    0.01      0.01      0.00     0.00  0.56  0.27   0.00  0.07   
1711   0.10    0.00      0.00      0.58     0.13  0.00  0.06   0.09  0.04   
1712   0.00    0.02      0.00      0.00     0.00  0.98  0.00   0.00  0.00   

      November  October  September  
0         0.00     0.00       0.00  
1

In [32]:
month_predicted_classes = month_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to month names
predicted_month_names = label_encoder_trans.inverse_transform(month_predicted_classes)
print(predicted_month_names)
print(month_predicted_classes)

['March' 'August' 'December' ... 'July' 'February' 'July']
[7 1 2 ... 5 3 5]


In [33]:
# Calculate accuracy
month_accuracy = accuracy_score(y_test, month_predicted_classes)
print("Accuracy:", month_accuracy)

Accuracy: 0.9305312317571512


In [34]:
# Calculate confusion matrix
month_conf_matrix = confusion_matrix(y_test, month_predicted_classes)
print("Confusion Matrix:")
print(month_conf_matrix)

Confusion Matrix:
[[125   0   0   0   0   0   0   7   8   0   0   0]
 [  0 177   0   0   0   2   0   0   0   0   0   1]
 [  0   0 115   0   0   0   0   0   0   8   1   0]
 [  6   0   0  65   8   0   0  19   0   0   0   0]
 [  1   0   0   8 143   0   0   3   0   0   0   0]
 [  0   2   0   0   0 176   0   0   0   0   0   0]
 [  0   0   0   0   0   0 132   0   1   0   0   0]
 [  6   0   0   7   0   0   0 105   1   0   0   0]
 [  7   0   0   0   0   0   1   0 143   0   0   0]
 [  0   0   4   0   0   0   0   0   0 106   7   0]
 [  0   0   0   0   0   0   0   0   0   2 148   3]
 [  0   3   0   0   0   0   0   0   0   0   3 159]]


In [35]:
# Calculate classification report (includes precision, recall, and F1-score for each class)
month_classification_rep = classification_report(y_test, month_predicted_classes, target_names=label_encoder_trans.classes_)
print("Classification Report:")
print(month_classification_rep)

Classification Report:
              precision    recall  f1-score   support

       April       0.86      0.89      0.88       140
      August       0.97      0.98      0.98       180
    December       0.97      0.93      0.95       124
    February       0.81      0.66      0.73        98
     January       0.95      0.92      0.93       155
        July       0.99      0.99      0.99       178
        June       0.99      0.99      0.99       133
       March       0.78      0.88      0.83       119
         May       0.93      0.95      0.94       151
    November       0.91      0.91      0.91       117
     October       0.93      0.97      0.95       153
   September       0.98      0.96      0.97       165

    accuracy                           0.93      1713
   macro avg       0.92      0.92      0.92      1713
weighted avg       0.93      0.93      0.93      1713



In [36]:
# Calculate AUC-ROC (requires one-hot encoded y_test and month_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
month_auc_roc = roc_auc_score(y_test_one_hot, month_probabilities, multi_class='ovr')
print("AUC-ROC:", month_auc_roc)

AUC-ROC: 0.9965563137404106


In [37]:
#get the month with the high mean of probability 
month_probabilities_df.describe(include='all')

,April,August,December,February,January,July,June,March,May,November,October,September
count,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000
mean,0.082201,0.107052,0.063433,0.058138,0.073024,0.101705,0.084588,0.079410,0.090887,0.071728,0.092662,0.095172
std,0.163644,0.226579,0.171790,0.132333,0.186877,0.223561,0.186054,0.159778,0.187489,0.167290,0.189468,0.200377
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.010000,0.020000,0.000000,0.000000,0.000000,0.020000,0.020000,0.010000,0.020000,0.000000,0.010000,0.010000
75%,0.070000,0.070000,0.020000,0.040000,0.020000,0.060000,0.060000,0.070000,0.070000,0.050000,0.070000,0.070000
max,0.920000,1.000000,0.940000,0.940000,0.970000,1.000000,0.980000,0.960000,0.970000,0.990000,0.960000,0.950000


From the above, we see the months of the year with the highest mean probability is August (0.106182) and July (0.101273). Therefore, the months of August and July should expect surgence in the crime occurance

Next we predict the probability for the hour

In [38]:
#hour of the day  
crime_data['OCC_HOUR'] = label_encoder_trans.fit_transform(crime_data['OCC_HOUR'])
target = crime_data['OCC_HOUR']

In [39]:
#Separate the different numerical,categorica features, target  
categorical_features = ['OCC_DOW','OCC_TIME_RANGE','OCC_MONTH']
numerical_features = ['OCC_DOY','OCC_DAY']
label = ['OCC_HOUR']
target = ['OCC_HOUR']

In [40]:
#Start with day of the week
# split data in train / test
X_train, X_test, y_train, y_test = train_test_split(crime_data[categorical_features + numerical_features]
                                                    ,crime_data[target],
                                                    test_size = 0.3, random_state=1234)

In [41]:
#define transformers and encoder
categorical_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
numerical_transformer = Pipeline(steps=[('scaler',StandardScaler())])

In [42]:
preprocessor = ColumnTransformer(transformers=[('categorical',categorical_transformer, categorical_features),
                                               ('numerical',numerical_transformer,numerical_features) ])

In [43]:
#define model
random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('regressor', RandomForestClassifier())])

In [44]:
random_forest.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OCC_DOW', 'OCC_TIME_RANGE',
                                                   'OCC_MONTH']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['OCC_DOY', 'OCC_DAY'])])),
                ('regressor', RandomForestClassifier())])

In [45]:
# Predict the probabilities of crime occurrence for each hour of the day in the test set
hour_probabilities = random_forest.predict_proba(X_test)

In [46]:
# Display the probabilities for each hour of the day in the test set
hour_names = label_encoder_trans.inverse_transform(random_forest.classes_)
hour_probabilities_df = pd.DataFrame(hour_probabilities, columns=hour_names)
print(hour_probabilities_df)

         0         1     2         3       4      5     6     7     8     9   \
0     0.000  0.000000  0.00  0.000000  0.0000  0.000  0.07  0.11  0.21  0.55   
1     0.000  0.000000  0.00  0.000000  0.0000  0.000  0.00  0.00  0.00  0.00   
2     0.000  0.000000  0.00  0.000000  0.0000  0.000  0.00  0.00  0.00  0.00   
3     0.100  0.314167  0.00  0.575833  0.0000  0.010  0.00  0.00  0.00  0.00   
4     0.195  0.082500  0.15  0.520000  0.0325  0.020  0.00  0.00  0.00  0.00   
...     ...       ...   ...       ...     ...    ...   ...   ...   ...   ...   
1708  0.000  0.000000  0.00  0.000000  0.0000  0.000  0.00  0.00  0.00  0.00   
1709  0.000  0.000000  0.00  0.000000  0.0000  0.000  0.01  0.00  0.00  0.00   
1710  0.000  0.000000  0.00  0.000000  0.0000  0.000  0.00  0.00  0.00  0.00   
1711  0.340  0.010000  0.58  0.000000  0.0700  0.000  0.00  0.00  0.00  0.00   
1712  0.030  0.010000  0.00  0.930000  0.0160  0.014  0.00  0.00  0.00  0.00   

      ...        14        15        16

In [47]:
hour_predicted_classes = hour_probabilities.argmax(axis=1)
# Inverse transform the numerical class labels back to hour names
predicted_hour_names = label_encoder_trans.inverse_transform(hour_predicted_classes)
print(predicted_hour_names)
print(hour_predicted_classes)

[ 9 12 18 ... 15  2  3]
[ 9 12 18 ... 15  2  3]


In [48]:
# Calculate accuracy
hour_accuracy = accuracy_score(y_test, hour_predicted_classes)
print("Accuracy:", hour_accuracy)

Accuracy: 0.19147694103911267


In [49]:
# Calculate confusion matrix
hour_conf_matrix = confusion_matrix(y_test, hour_predicted_classes)
print("Confusion Matrix:")
print(hour_conf_matrix)

Confusion Matrix:
[[72 32 34 34 18 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [34 20 35 12  9  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [38 15 28 21 10  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [42  8 19 15  9  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [19  8 13 14  4  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [15  2  8  7  5  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  7  1  0  3  6  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  3  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  3  3  5  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  3  3  2  5  3  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  3  4  2  3  2  6  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1 10  1  7  7  5  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  4  6  2  1  6 10  0  0  0  0  0  0]
 [ 0  0

In [50]:
# Calculate AUC-ROC (requires one-hot encoded y_test and hour_probabilities)
y_test_one_hot = pd.get_dummies(y_test)
hour_auc_roc = roc_auc_score(y_test_one_hot, hour_probabilities, multi_class='ovr')
print("AUC-ROC:", hour_auc_roc)

AUC-ROC: 0.8546428583633107


In [51]:
#get the day with the high mean of probability 
hour_probabilities_df.describe(include='all')

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,...,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000,1713.000000
mean,0.117690,0.053705,0.080899,0.062040,0.035244,0.020894,0.010181,0.010537,0.006549,0.012291,...,0.023614,0.026357,0.025016,0.039457,0.039853,0.058691,0.057759,0.080055,0.083591,0.089741
std,0.227787,0.143406,0.182081,0.154500,0.116672,0.088925,0.057486,0.061035,0.039238,0.066471,...,0.091739,0.100772,0.097028,0.131253,0.121397,0.149049,0.147018,0.178514,0.182453,0.192515
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.105000,0.020000,0.040000,0.021429,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.025000,0.030000,0.048333,0.050000,0.055500
max,1.000000,0.973333,0.930000,0.930000,0.923333,0.730000,0.840000,0.740000,0.550000,0.770000,...,0.780000,0.860000,0.910000,0.780000,0.850000,0.950000,0.920000,0.936667,0.970000,0.915833


From the above, we see the hour of the day with the highest mean probability is midnight - 00:00 (0.117828) and 23:00 (0.090266). Therefore, the hours of 23:00 to midnight should expect surgence in the crime occurance

From our model we can say that during the weekends (saturday and sunday) of July and August in the time of 23:00 to midnight, the NCOP should expect occurance of crimes and should dedicate more planning towards this time period